# Part 1: Stand/Sit Detector
### This part creates a stand/sit detector based on images of sheep and uses accelerometer data as ground truth

In [1]:
# imports
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from tensorflow.keras import layers, models
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
pre_loaded = True

In [2]:
if not pre_loaded:
    # read in the data first
    files = ['W.csv', 'B.csv', 'M.csv', 'P.csv', 'O_0.csv', 'N.csv', 'K.csv', 'Q.csv']
    sheep_raw = []
    data_dir = 'data/'
    for file in files:
        path = data_dir + file
        df = pd.read_csv(path)
        # drop unnecessary columns
        df = df.drop(columns=df.columns[3:])
        df = df.drop(columns=df.columns[0:1])

        # rename columns
        df.columns = ['File', 'Standing']

        # change accelerometer value to 1 if standing, 0 if sitting
        df['Standing'] = np.where(df['Standing'] < -0.4, 1, 0)

        # change the timestamp to the filename for the image
        time_stamps = df['File'].to_list()
        file_names = []
        for ts in time_stamps:
            filename = ts
            filename = filename.replace(' ', '_')
            filename = filename.replace(':', ',')
            filename = filename.replace('/', '-')
            filename = filename + '.png'

            # UNCOMMENT THIS LINE WHEN YOU KNOW PATH TO IMAGE FOLDER
            filename = '/mnt/d/WhitlockSheepStudy/output' + '/' + file[0] + '/' + filename

            file_names.append(filename)

        df = df.assign(File=file_names)
        sheep_raw.append(df)

In [ ]:
print(os.listdir('/mnt/d/WhitlockSheepStudy/output'))

In [3]:
if not pre_loaded:
    # Grab images from files on the hard drive
    X = []
    y = []

    bad_reads = 0
    for sheep in sheep_raw:
        files = sheep['File'].to_list()
        labels = sheep['Standing'].to_list()
        for file, label in zip(files, labels):
            try:
                img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
                # downscale image
                img = cv2.resize(img, (108,243))

                # normalize image
                img = img/255
                # img = cv2.normalize(img, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

                # store image
                X.append(img)
                y.append(label)

                # save every 100 images
                if len(X) % 100 == 0:
                    np.save('X.npy', np.asarray(X))
                    np.save('y.npy', np.asarray(y))
                    print('Saving...')
                
                if len(X) % 1000 == 0:
                    break
            except:
                bad_reads += 1
                continue

    print(f'Bad or missing images: {bad_reads}')
    print(f'Good images:           {len(X)}')

In [4]:
if pre_loaded:
    X = np.load('X.npy')
    y = np.load('y.npy')

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [6]:
def create_model(num_filters: list, kernel_sizes: list, input_shape: tuple, optimizer: str):
    model = models.Sequential()

    model.add(layers.Conv2D(num_filters[0], (kernel_sizes[0],kernel_sizes[0]), activation='relu', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2,2)))

    for i in range(1, len(kernel_sizes)):
        model.add(layers.Conv2D(num_filters[i], (kernel_sizes[i],kernel_sizes[i]), activation='relu'))
        model.add(layers.MaxPooling2D((2,2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
    # print(model.summary())

    return model

In [7]:
params = {
    'num_filters': [64, 32, 16],
    'kernel_sizes': [3, 3, 3]
}

X_train_np = np.array(X_train)
X_train_np = X_train_np.reshape(X_train_np.shape[0], X_train_np.shape[1], X_train_np.shape[2], 1)
X_test_np = np.array(X_test)
X_test_np = X_test_np.reshape(X_test_np.shape[0], X_test_np.shape[1], X_test_np.shape[2], 1)

y_train_np = np.array(y_train)
y_test_np = np.array(y_test)

model = create_model(params['num_filters'], params['kernel_sizes'], X_train_np[0].shape, 'sgd')

In [8]:
del X_train
del X_test
del y_train
del y_test

In [30]:
print(type(X_train_np[0][0][0][0]))

<class 'numpy.float64'>


In [ ]:
# # create tf dataset to save space
# train_dataset = tf.data.Dataset.from_tensor_slices((X_train_np, y_train_np))
# test_dataset = tf.data.Dataset.from_tensor_slices((X_test_np, y_test_np))

In [11]:
# train the model
es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

history = model.fit(X_train_np, y_train_np, epochs=50, validation_data=(X_test_np, y_test_np), batch_size=16, callbacks=[es])

Epoch 1/10
400/400 [==============================] - 20s 24ms/step - loss: 0.6778 - accuracy: 0.5911 - val_loss: 0.6478 - val_accuracy: 0.7013
Epoch 2/10
400/400 [==============================] - 9s 23ms/step - loss: 0.6157 - accuracy: 0.6755 - val_loss: 0.5861 - val_accuracy: 0.7206
Epoch 3/10
400/400 [==============================] - 9s 24ms/step - loss: 0.5680 - accuracy: 0.7077 - val_loss: 0.5354 - val_accuracy: 0.7419
Epoch 4/10
400/400 [==============================] - 10s 24ms/step - loss: 0.5264 - accuracy: 0.7398 - val_loss: 0.5035 - val_accuracy: 0.7556
Epoch 5/10
400/400 [==============================] - 10s 25ms/step - loss: 0.4822 - accuracy: 0.7722 - val_loss: 0.4750 - val_accuracy: 0.7856
Epoch 6/10
400/400 [==============================] - 18s 45ms/step - loss: 0.4448 - accuracy: 0.8036 - val_loss: 0.4134 - val_accuracy: 0.8225
Epoch 7/10
400/400 [==============================] - 40s 99ms/step - loss: 0.4055 - accuracy: 0.8242 - val_loss: 0.3965 - val_accuracy: 0

In [12]:
# save the model
model.save('models/ten_epoch_SGD')

INFO:tensorflow:Assets written to: models/ten_epoch_SGD/assets


INFO:tensorflow:Assets written to: models/ten_epoch_SGD/assets


In [9]:
# Time to do a grid search
#num_filters: list, kernel_sizes: list, input_shape: tuple, optimizer: str

num_filters = [
    [32, 16, 8],
    [64, 32, 16],
]
kernel_sizes = [
    [3, 3, 3],
    [2, 2, 2]
]
optimizers = ['sgd', 'adam', 'rmsprop']

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

for i, filters in enumerate(num_filters):
    for j, kernels in enumerate(kernel_sizes):
        for k, opt in enumerate(optimizers):
            if (i == 0 and j == 0 and k == 0) or (i == 0 and j == 0 and k == 1):
                continue
            model = create_model(filters, kernels, X_train_np[0].shape, opt)

            log_dir = f'logs/model_{i}_{j}_{k}'
            tb = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

            model.fit(X_train_np, y_train_np, epochs=50, validation_data=(X_test_np, y_test_np), batch_size=16, callbacks=[tb, es])
            model.save(f'models/model_{i}_{j}_{k}')
            

Epoch 1/50
400/400 [==============================] - 21s 50ms/step - loss: 0.5462 - accuracy: 0.7195 - val_loss: 0.3831 - val_accuracy: 0.8525
Epoch 2/50
400/400 [==============================] - 21s 53ms/step - loss: 0.2996 - accuracy: 0.8788 - val_loss: 0.2286 - val_accuracy: 0.9181
Epoch 3/50
400/400 [==============================] - 18s 45ms/step - loss: 0.1896 - accuracy: 0.9311 - val_loss: 0.1678 - val_accuracy: 0.9525
Epoch 4/50
400/400 [==============================] - 18s 45ms/step - loss: 0.1455 - accuracy: 0.9495 - val_loss: 0.1501 - val_accuracy: 0.9544
Epoch 5/50
400/400 [==============================] - 19s 47ms/step - loss: 0.1163 - accuracy: 0.9605 - val_loss: 0.1518 - val_accuracy: 0.9619
Epoch 6/50
400/400 [==============================] - 18s 44ms/step - loss: 0.0947 - accuracy: 0.9709 - val_loss: 0.1990 - val_accuracy: 0.9425
Epoch 7/50
400/400 [==============================] - 17s 43ms/step - loss: 0.0853 - accuracy: 0.9708 - val_loss: 0.1184 - val_accuracy:

INFO:tensorflow:Assets written to: models/model_0_0_2/assets


INFO:tensorflow:Assets written to: models/model_0_0_2/assets


Epoch 1/50
400/400 [==============================] - 19s 46ms/step - loss: 0.6855 - accuracy: 0.5616 - val_loss: 0.6778 - val_accuracy: 0.6488
Epoch 2/50
400/400 [==============================] - 17s 41ms/step - loss: 0.6573 - accuracy: 0.6577 - val_loss: 0.6253 - val_accuracy: 0.6944
Epoch 3/50
400/400 [==============================] - 15s 38ms/step - loss: 0.6045 - accuracy: 0.6958 - val_loss: 0.5747 - val_accuracy: 0.7038
Epoch 4/50
400/400 [==============================] - 16s 40ms/step - loss: 0.5626 - accuracy: 0.7125 - val_loss: 0.5399 - val_accuracy: 0.7350
Epoch 5/50
400/400 [==============================] - 16s 39ms/step - loss: 0.5255 - accuracy: 0.7336 - val_loss: 0.5093 - val_accuracy: 0.7444
Epoch 6/50
400/400 [==============================] - 16s 39ms/step - loss: 0.4884 - accuracy: 0.7702 - val_loss: 0.4628 - val_accuracy: 0.8181
Epoch 7/50
400/400 [==============================] - 15s 39ms/step - loss: 0.4460 - accuracy: 0.8034 - val_loss: 0.4490 - val_accuracy:

INFO:tensorflow:Assets written to: models/model_0_1_0/assets


...

INFO:tensorflow:Assets written to: models/model_1_1_1/assets


INFO:tensorflow:Assets written to: models/model_1_1_1/assets


Epoch 1/50
400/400 [==============================] - 36s 89ms/step - loss: 0.5654 - accuracy: 0.6956 - val_loss: 0.4455 - val_accuracy: 0.8169
Epoch 2/50
400/400 [==============================] - 33s 83ms/step - loss: 0.3332 - accuracy: 0.8672 - val_loss: 0.2606 - val_accuracy: 0.8981
Epoch 3/50
400/400 [==============================] - 32s 81ms/step - loss: 0.2015 - accuracy: 0.9291 - val_loss: 0.2449 - val_accuracy: 0.9100
Epoch 4/50
400/400 [==============================] - 32s 81ms/step - loss: 0.1414 - accuracy: 0.9539 - val_loss: 0.3080 - val_accuracy: 0.8888
Epoch 5/50
400/400 [==============================] - 33s 81ms/step - loss: 0.1159 - accuracy: 0.9647 - val_loss: 0.1496 - val_accuracy: 0.9513
Epoch 6/50
400/400 [==============================] - 33s 82ms/step - loss: 0.0942 - accuracy: 0.9689 - val_loss: 0.1451 - val_accuracy: 0.9631
Epoch 7/50
400/400 [==============================] - 32s 81ms/step - loss: 0.0828 - accuracy: 0.9731 - val_loss: 0.1203 - val_accuracy:

INFO:tensorflow:Assets written to: models/model_1_1_2/assets


INFO:tensorflow:Assets written to: models/model_1_1_2/assets
